Single bead analysis
====================

This example shows how you can use PyAdditive to determine melt pool
characteristics for a given material and machine parameter combinations.

Units are SI (m, kg, s, K) unless otherwise noted.


Perform required imports and connect
====================================

Perform the required imports and connect to the Additive service.


In [ ]:
import matplotlib.pyplot as plt

from ansys.additive.core import (
    Additive,
    AdditiveMachine,
    MeltPoolColumnNames,
    SimulationError,
    SingleBeadInput,
)

additive = Additive()

Get server connection information
=================================

Get server connection information using the
`about <Additive.about>`{.interpreted-text role="meth"} method.


In [ ]:
print(additive.about())

Select material
===============

Select a material. You can use the
`get_materials_list() <ansys.additive.core.additive.Additive.get_materials_list>`{.interpreted-text
role="meth"} method to obtain a list of available materials.


In [ ]:
print(additive.get_materials_list())

You can obtain the parameters for a single material by passing a name
from the materials list to the
`get_material() <ansys.additive.core.additive.Additive.get_material>`{.interpreted-text
role="meth"} method.


In [ ]:
material = additive.get_material("17-4PH")

Specify machine parameters
==========================

Specify machine parameters by first creating an
`AdditiveMachine <from ansys.additive.core.machine.AdditiveMachine>`{.interpreted-text
role="class"} object then assigning the desired values. All values are
in SI units (m, kg, s, K) unless otherwise noted.


In [ ]:
machine = AdditiveMachine()

# Show available parameters
print(machine)

Set laser power and scan speed
==============================

Set the laser power and scan speed.


In [ ]:
machine.scan_speed = 1  # m/s
machine.laser_power = 300  # W

Specify inputs for single bead simulation
=========================================

Create a
`SingleBeadInput <ansys.additive.core.single_bead.SingleBeadInput>`{.interpreted-text
role="class"} object containing the desired simulation parameters.


In [ ]:
input = SingleBeadInput(
    machine=machine, material=material, id="single-bead-example", bead_length=0.0012  # meters
)

Run simulation
==============

Use the
`simulate() <ansys.additive.core.additive.Additive.simulate>`{.interpreted-text
role="meth"} method of the `additive` object to run the simulation. The
returned object is a either a
`SingleBeadSummary <ansys.additive.core.single_bead.SingleBeadSummary>`{.interpreted-text
role="class"} object containing the input and a
`MeltPool <ansys.additive.core.single_bead.MeltPool>`{.interpreted-text
role="class"} or a :class:\`SimulationError
\<ansys.additive.core.simulation.SimulationError\>.


In [ ]:
summary = additive.simulate(input)
if isinstance(summary, SimulationError):
    raise Exception(summary.message)

Plot melt pool statistics
=========================

Obtain a `Pandas DataFrame <pandas.DataFrame>`{.interpreted-text
role="class"} containing the melt pool statistics by using the
`data_frame() <ansys.additive.core.single_bead.MeltPool.data_frame>`{.interpreted-text
role="meth"} property of the `melt_pool` attribute of the `summary`
object. Use the
`DataFrame.plot() <pandas.DataFrame.plot>`{.interpreted-text
role="meth"} method to plot the melt pool dimensions as a function of
bead length.


In [ ]:
df = summary.melt_pool.data_frame().multiply(1e6)  # convert from meters to microns
df.index *= 1e3  # convert bead length from meters to millimeters

df.plot(
    y=[
        MeltPoolColumnNames.LENGTH,
        MeltPoolColumnNames.WIDTH,
        MeltPoolColumnNames.DEPTH,
        MeltPoolColumnNames.REFERENCE_WIDTH,
        MeltPoolColumnNames.REFERENCE_DEPTH,
    ],
    ylabel="Melt Pool Dimensions (µm)",
    xlabel="Bead Length (mm)",
    title="Melt Pool Dimensions vs Bead Length",
)
plt.show()

List melt pool statistics
=========================

You can show a table of the melt pool statistics by typing the name of
the data frame object and pressing enter. For brevity, the following
code uses `head()` so that only the first few rows are shown.


In [ ]:
df.head()

# .. note::
#    If running this example as a Python script, no output is shown.

Save melt pool statistics
=========================

Save the melt pool statistics to a CSV file using the
`to_csv() <pandas.DataFrame.to_csv>`{.interpreted-text role="meth"}
method.


In [ ]:
df.to_csv("melt_pool.csv")